# Quick start and SQL query example
This notebook shows how to use the pipeline and some example queries that can be made from the database.

In [2]:
from carbon_stats import CarbonStats
import sqlite3
import pandas as pd
import geopandas as gpd

# Run the Pipeline and Populate the Database
Using the pipeline defined in carbon_stats.py, the following cell shows an example of running the pipeline.
This cell with takes 1 minute or more to run, mostly because the pipeline currently uses raster_stats.zonal_stats, which is simple to implement but slow with large rasters.  

In [3]:
# Where the database file will be saved
db_file = "data/outputs/county_carbon.sqlite"

# Initialize the processor
cs = CarbonStats(
    raster_file="data/inputs/NLS_TotalEcosystemCarbon2020.tif",
    boundary_file="data/inputs/US_census_counties/cb_2024_us_county_500k.shp"
)

# Process the data
cs.read_raster()
cs.process_boundaries(['Michigan', 'Wisconsin', 'Minnesota'])
cs.calculate_stats()

# Export to database
cs.export_to_sqlite(db_file)

Successfully exported to data/outputs/county_carbon.sqlite


'data/outputs/county_carbon.sqlite'

# Example SQL query
Here's an example of reading from the database we just created.  Then, if we like, we can import it back to a GeoDataFrame.

In [6]:
#Connect to SQLite database file
conn = sqlite3.connect(db_file)

# Query all columns for Minnesota counties with more than 50 Tg CO2e
# Order from lowest to highest total_Tg_CO2e
query = """
SELECT *
FROM county_carbon
WHERE state_name = 'Minnesota'
  AND total_Tg_CO2e > 50
ORDER BY total_Tg_CO2e; 
"""

# Use pandas to execute query and load into DataFrame
df_from_sql = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Transform df to a GeoDataFrame
gdf_from_sql = gpd.GeoDataFrame(
    df_from_sql,
    geometry=gpd.GeoSeries.from_wkt(df_from_sql['geometry_wkt_EPSG_4326']),
    crs='EPSG:4326'
)
gdf_from_sql = gdf_from_sql.drop(columns='geometry_wkt_EPSG_4326')

# Output the GeoDataFrame to verify
gdf_from_sql.head()

,geoid,county_name,state_name,total_Tg_CO2e,average_Mg_CO2e_per_acre,county_area_acres,geometry
0,27135,Roseau,Minnesota,59.777109,55.656440,1.074038e+06,"POLYGON ((-96.40541 48.99998, -96.20883 48.999..."
1,27097,Morrison,Minnesota,60.825771,82.391388,7.382540e+05,"POLYGON ((-94.65346 46.34868, -94.65079 46.347..."
2,27111,Otter Tail,Minnesota,77.213624,54.236593,1.423644e+06,"POLYGON ((-96.2812 46.63078, -96.24015 46.6308..."
3,27017,Carlton,Minnesota,78.313274,139.839912,5.600209e+05,"POLYGON ((-93.0647 46.68009, -93.06457 46.6836..."
4,27029,Clearwater,Minnesota,79.071139,119.910225,6.594195e+05,"POLYGON ((-95.58289 48.02067, -95.22911 48.020..."
